In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import os
from keras.models import load_model

In [ ]:
import cobra
%load_ext autoreload
import sys
if not '/home/nlarusstone/cf_fba' in sys.path:
    sys.path.append('/home/nlarusstone/cf_fba')
import src.utils as utils
import src.flux_utils as futils
import src.create_dataset as dataset
import src.flux_sample as fs
from src.get_test_data import get_test_data
import src.cf_io as cf_io
import src.correlation as corr
%autoreload 2

In [ ]:
for f in os.listdir('../results'):
    print f
    res_df = pd.read_csv('../results/' + f)
    #print res_df.head()
    res_df.drop('Unnamed: 0', inplace=True, axis=1)
    print scipy.stats.pearsonr(res_df.iloc[:, 0], res_df.iloc[:, 1])

In [ ]:
latent_dim_l = [2, 10]
use_corr = True
froot_l = ['manual', 'echo', 'karim']
if froot == 'karim':
    dset = 'karim'
else:
    dset = 'nls'
txtl_l = [False, True]
resamp = True
layer_szs_l = [[1024, 1024, 1024], [1024, 512, 256]]
model = 'iJO1366'

In [ ]:
%%script bash --bg
echo 'hi'

In [ ]:
from multiprocessing import Process, Pipe

In [ ]:
res_l = []
bad_rxns_l = []
best_l = []
for froot in froot_l:
    df = cf_io.get_exp_data(froot)
    for latent_dim in latent_dim_l:
        for layer_szs in layer_szs_l:
            for txtl in txtl_l:
                encoder, generator, X_test, y_test, obj_col, cols, y_vals_d, test_enc, test_dec = get_test_data(froot, txtl, resamp, latent_dim, layer_szs, use_corr=True)
                cols = np.array(cols)
                model_f = '../bio_models/{0}/{1}/{2}_cf{3}.sbml'.format(dset, froot, model, '_txtl' if txtl else '')
                #res, bad_rxns, best = corr.optimized_corr(model_f, df, test_dec, cols, t_low=-3, t_high=-1, n_steps=30)
                res_l.append(res)
                bad_rxns_l.append(bad_rxns)
                best_l.append(best)
                break
            break
        break
    break

In [ ]:
bad = np.zeros((1, X_test.shape[1], X_test.shape[2]))
bad_enc = encoder.predict(bad)

In [ ]:
bad_dec = generator.predict(bad_enc)

In [ ]:
bad_enc

In [ ]:
bad_dec.shape

In [ ]:
model_f = '../bio_models/{0}/{1}/{2}_cf{3}.sbml'.format(dset, froot, model, '_txtl' if args.txtl else '')

In [ ]:
res, bad_rxns, best = corr.optimized_corr(model_f, df, test_dec, cols, t_low=-2, t_high=-1, n_steps=30)

In [ ]:
best[1]

In [ ]:
test_dec[:, :, :].mean().shape

In [ ]:
mod = cobra.io.read_sbml_model('../bio_models/ColiPrunedComp_wout.xml')
mod

In [ ]:
for t in thresh:
    print t
    #bad_cols = cols[dec_df.mean(axis=0).mean(axis=0) < t]
    #bad_cols = cols[np.random.choice(range(len(cols)), size=300)]
    #bad_cols = bad_cols[bad_cols != obj_name]
    objs = []
    #bad_rxns = []
    for idx, row in df.iterrows():
        model_f_i = model_f_base[0] + '_{0}.'.format(idx) + model_f_base[1]
        model_i = cobra.io.read_sbml_model(model_f_i)
        #bad_rxns.append(bad_cols)
        #model_i.remove_reactions(bad_cols)
        sol = model_i.optimize()
        break
    break

In [ ]:
#def optimized_corr(model_f, df, test_dec, cols):
np.random.seed(42)
thresh = np.logspace(start=t_low, stop=t_high, num=30)
bad_rxns_t = []
res = []
best = (0, None)
dec_df = test_dec[:, :, :]#pd.DataFrame(data=test_dec[:, :, :], columns=cols)
model_f_base = model_f.rsplit('.', 1)
mod = cobra.io.read_sbml_model(model_f_base[0] + '_0.' + model_f_base[1])
obj_name = str(mod.objective.expression.args[0]).split('*')[1]
for t in thresh:
    print t
    bad_cols = cols[dec_df.mean(axis=0).mean(axis=0) < t]
    bad_cols = cols[np.random.choice(range(len(cols)), size=300)]
    bad_cols = bad_cols[bad_cols != obj_name]
    objs = []
    #bad_rxns = []
    for idx, row in df.iterrows():
        model_f_i = model_f_base[0] + '_{0}.'.format(idx) + model_f_base[1]
        model_i = cobra.io.read_sbml_model(model_f_i)
        #bad_rxns.append(bad_cols)
        model_i.remove_reactions(bad_cols)
        sol = model_i.optimize()
        objs.append(sol.objective_value)
    bad_rxns_t.append(bad_cols)
    corr = scipy.stats.pearsonr(objs, df['OUT'])
    print corr
    if corr[0] > best[0]:
        best = (corr[0], bad_cols)
    res.append(objs)
print 'Best: {0}'.format(best[0])
#return pd.DataFrame(res), pd.DataFrame(bad_rxns_t), pd.DataFrame(best[1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
X_test[:, :, :].mean(axis=-1).shape

In [ ]:
fig, axarr = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))
ax1, ax2 = axarr
ax1.hist(X_test[:, :, :].mean(axis=0).mean(axis=0))
ax1.set_title('Distribution of average reactions fluxes')
ax1.set_ylabel('Number of reactions')
ax1.set_xlabel('Normalized flux value')

ax2.hist(X_test[:, :, :].mean(axis=-1).mean(axis=-1))
ax2.set_title('Distribution of average fluxes across samples')
ax2.set_ylabel('Number of samples')
ax2.set_xlabel('Normalized flux value')

In [ ]:
mod.reactions.query('biomass')

In [ ]:
model_f = '{0}_ecoli_cf_base{1}.sbml'.format(dset, '_txtl' if txtl else '')
test = '{0}_ecoli_cf_base{1}_0.sbml'.format(dset, '_txtl' if txtl else '')
if not os.path.exists('../models/{0}/{1}'.format(froot, test)):
    gen_models()
    
thresh = np.logspace(start=-2, stop=-1, num=20)
bad_rxns_t = []
res = []
for t in thresh:
    print t
    objs = []
    bad_rxns = []
    for idx, row in df.iterrows():
        model_f_i = '{0}_ecoli_cf_base{1}_{2}.sbml'.format(dset, '_txtl' if txtl else '', idx)
        model_i = cobra.io.read_sbml_model('../models/{0}/'.format(froot) + model_f_i)
        obj_name = str(model_i.objective.expression.args[0]).split('*')[1]
        dec_df = pd.DataFrame(data=test_dec[:, idx, :], columns=cols)
        bad_cols = cols[dec_df.mean() < t]
        bad_cols = bad_cols[bad_cols != obj_name]
        bad_rxns.append(bad_cols)
        model_i.remove_reactions(bad_cols)
        sol = model_i.optimize()
        objs.append(sol.objective_value)
    bad_rxns_t.append(bad_rxns)
    print scipy.stats.pearsonr(objs, df['OUT'])
    res.append(objs)

In [ ]:
rxn_amt = 5
addl_amt = 1
batch_size = 50
n_batches = batch_size / rxn_amt
model_f = '{0}_ecoli_cf_base{1}.sbml'.format('nls', '_txtl' if txtl else '')

print 'Read in data'
df = fs.get_exp_data(froot)
#df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)

cfps_conc = pd.read_csv('../data/{0}_concs'.format('nls'), index_col='compound')
if not froot == 'karim':
    cfps_conc.drop('final_conc', inplace=True, axis=1)
    nrg_mix = pd.read_csv('../data/energy_mix.csv', index_col='compound')
else:
    nrg_mix = None
#print cfps_conc

print 'Time to generate model specific conditions'
model = cobra.io.read_sbml_model('../models/{0}'.format(model_f))
print 'Model {0} read in'.format(model_f)

In [ ]:
t = 10 ** -4.58
pd.DataFrame(data=test_dec[:, 0, :], columns=cols)
bad_cols = cols[dec_df.mean() < t]
bad_cols

In [ ]:
res = []
thresh = np.logspace(start=-2, stop=-1, num=20)
for t in thresh:
    print t
    objs = []
    for idx, row in df.iterrows():
        print idx
        cfps_conc_tmp = cfps_conc.copy()
        cfps_conc_tmp = fs.update_vals(cfps_conc_tmp, row, n_batches, nrg_mix=nrg_mix, replace=args.froot == 'karim')
        tot_rxn_amt = rxn_amt + addl_amt
        if args.froot == 'karim':
            exp_concs = cfps_conc_tmp['final_conc']
        else:
            exp_concs = fs.conc_dilution(cfps_conc_tmp['start_conc'], (rxn_amt / 5.0) * cfps_conc_tmp['amt'], tot_rxn_amt)
        model_i = fs.change_conc(model, exp_concs)
        dec_df = pd.DataFrame(data=test_dec[:, idx, :], columns=cols)
        bad_cols = cols[dec_df.mean() < t]
        bad_cols = bad_cols[bad_cols != 'DM_PROTEIN_RFP']
        model_i.remove_reactions(bad_cols)
        #print model_i.reactions.EX_mg2_c.bounds
        #print model_i.reactions.EX_pi_c.bounds
        sol = model_i.optimize()
        objs.append(sol.objective_value)
    print scipy.stats.pearsonr(objs, df['OUT'])
    res.append(objs)
#res_df = pd.DataFrame(res)
#res_df['Actual'] = df['OUT']
#res_df.to_csv('../results/{0}_{1}_unoptimized_results{2}'.format(args.froot, args.dataset, '_txtl' if args.txtl else ''))

In [ ]:
for r in res:
    print r
    print scipy.stats.pearsonr(r, df['OUT'])

In [ ]:
#t = 0.016237767391887217
t = 0.01
bad_rxns = []
objs = []
for idx, row in df.iterrows():
    print idx
    cfps_conc_tmp = cfps_conc.copy()
    cfps_conc_tmp = fs.update_vals(cfps_conc_tmp, row, n_batches, nrg_mix=nrg_mix, replace=args.froot == 'karim')
    tot_rxn_amt = rxn_amt + addl_amt
    if args.froot == 'karim':
        exp_concs = cfps_conc_tmp['final_conc']
    else:
        exp_concs = fs.conc_dilution(cfps_conc_tmp['start_conc'], (rxn_amt / 5.0) * cfps_conc_tmp['amt'], tot_rxn_amt)
    model_i = fs.change_conc(model, exp_concs)
    dec_df = pd.DataFrame(data=test_dec[:, idx, :], columns=cols)
    bad_cols = cols[dec_df.mean() < t]
    bad_cols = bad_cols[bad_cols != 'DM_PROTEIN_RFP']
    bad_rxns.append(bad_cols)
    model_i.remove_reactions(bad_cols)
    #print model_i.reactions.EX_mg2_c.bounds
    #print model_i.reactions.EX_pi_c.bounds
    sol = model_i.optimize()
    objs.append(sol.objective_value)
print scipy.stats.pearsonr(objs, df['OUT'])

In [ ]:
prot = model.reactions

In [ ]:
brs = bad_rxns[0]
prot = model.reactions
for br in bad_rxns:
    print len(br)
    for r in br:
        if r in prot:
            prot.remove(r)
    brs = np.intersect1d(brs, br)

In [ ]:
prot

In [ ]:
m = cobra.Model('cfps')

In [ ]:
m.add_reactions(prot)

In [ ]:
m.objective = m.reactions.DM_PROTEIN_RFP

In [ ]:
m.optimize()
m.summary()